In [ ]:
# This is a notebook for model validation, for my own purposes.

In [1]:
import os
import sys
import numpy as np
import lasagne as L

sys.path.append('../')
sys.path.append('../scripts/')

from squad_tools import load_glove, load_squad_dev
from os.path import join
from QANet import QANet

Using cuDNN version 5105 on context None
Preallocating 3661/12204 Mb (0.300000) on cuda

floatX == float32
device == cuda



Mapped name None to device cuda: GeForce GTX TITAN X (0000:01:00.0)


In [2]:
glove_path = '/pio/data/data/glove_vec/6B/glove.6B.300d.txt'
squad_base_path = '/pio/data/data/squad/'
squad_path = join(squad_base_path, 'preproc')

glove_words, glove_embs = load_glove(glove_path)
voc_size = glove_embs.shape[0]
NAW_token = glove_words.index('<not_a_word>')

# Weryfikacja modeli

In [6]:
%%time
negative = True  # whether or not to use negative model
train_unk = True

if not negative:
    model_fname = '../models/best/model.ep09.npz'
    conv = 'valid'
else:
    # this was trained with squad-rng, wiki-neg, wiki-pos
    model_fname = '../models/model_neg03/model.ep07.npz'
    conv = 'full'
    
dev_data = load_squad_dev(
    squad_base_path, 
    squad_path, 
    lower_raw=True,
    make_negative=negative, 
    NAW_token=NAW_token)

qa_net = QANet(
    voc_size=voc_size,
    dev_data=dev_data,
    emb_init=glove_embs,
    skip_train_fn=True,
    train_unk=train_unk,
    negative=negative,
    conv=conv)

qa_net.load_params(model_fname)

Adding NAW token to dev set.
Building the model...
Using dropout after wiq calculation.
Compiling theano functions:
    get_intermediate_results_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 14.1 s, sys: 332 ms, total: 14.4 s
Wall time: 17.4 s


In [5]:
%%time
# with negative=False (default)
qa_net.calc_dev_f1(0)

Calculating validation F1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
Getting answers...
Done
F1:  72.34538298
EM:  63.1031220435
CPU times: user 46.5 s, sys: 688 ms, total: 47.2 s
Wall time: 47.5 s


72.34538298000702

In [7]:
%%time
# with negative=True (F1 neg)
qa_net.calc_dev_f1(0)

Calculating validation F1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
Getting answers...
Done
F1:  57.8313072749
EM:  50.8514664144
CPU times: user 49 s, sys: 700 ms, total: 49.7 s
Wall time: 49.6 s


57.8313072748809